# Comparação Experimental de Técnicas de Classificação

Este trabalho tem como objetivo realizar uma **comparação experimental** entre um conjunto pré-definido de técnicas de aprendizado e classificação automática aplicadas a um problema de **classificação supervisionada**.

## Técnicas Utilizadas

As seguintes técnicas de aprendizado serão avaliadas:

- **Decision Tree (DT)**
- **K Nearest Neighbors (KNN)**
- **Multi-layer Perceptron (MLP)**
- **Random Forest (RF)**
- **Heterogeneous Boosting (HB)**

## Procedimento Experimental

O experimento será conduzido em **3 rodadas** de ciclos aninhados de validação e teste, organizados da seguinte forma:

- **Validação interna:** 4 folds
- **Teste externo:** 10 folds

A seleção de hiperparâmetros será realizada por **busca em grade** (_grid search_) no ciclo interno, com os seguintes valores para cada técnica:

### Hiperparâmetros

```python
# Decision Tree (DT)
{
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15, 25]
}

# K Nearest Neighbors (KNN)
{
    'n_neighbors': [1, 3, 5, 7, 9]
}

# Multi-layer Perceptron (MLP)
{
    'hidden_layer_sizes': [(100,), (10,)],
    'alpha': [0.0001, 0.005],
    'learning_rate': ['constant', 'adaptive']
}

# Random Forest (RF)
{
    'n_estimators': [5, 10, 15, 25],
    'max_depth': [10, None]
}

# Heterogeneous Boosting (HB)
{
    'n_estimators': [5, 10, 15, 25, 50]
}


# Imports

In [ ]:
# Manipulação de dados
import numpy as np
import pandas as pd

# Modelos de classificação
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier


# Validação cruzada e avaliação
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

# Pré-processamento
from sklearn.preprocessing import MinMaxScaler

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Criação do Heterogeneous Boosting
from collections import Counter

from tqdm import tqdm
from sklearn.base import BaseEstimator, ClassifierMixin, clone
import numpy as np
from scipy.stats import mode
from sklearn.naive_bayes import GaussianNB

from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from scipy.stats import t


# Configurações de exibição

In [ ]:
# Configurações gerais de visualização
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
warnings.filterwarnings("ignore")

# Importando base de dados

In [ ]:
data = pd.read_csv("jogosLoL2021.csv")

In [ ]:
data

# Pré-processamento dos dados

**Descartar o identificador da partida** e realizar a **padronização das características numéricas** (normalização).

As características que usaremos são os dados pré-jogos, ou seja, as informações disponíveis antes do início da partida, como:

- WR (Win-Rate do Time Azul)
- KD (Kill-to-Death Ratio do Time Azul)
- GPR (Gold Percent Ratio do Time Azul)
- GSPD (Average Gold Spent Ratio do Time Azul)
- EGR (Early-Game-Rate do Time Azul)
- MLR (Mid-Late-Game-Rate do Time Azul)
- FB% (First Blood Rate do Time Azul)
- FT% (First Blood Rate do Time Azul)
- F3T% (First To Three Towers Rate do Time Azul)
- HLD% (Harold Rate do Time Azul)
- DRG% (Dragon Rate do Time Azul)
- BN% (First Blood Rate do Time Azul)
- LNE% (Lane Control Rate do Time Azul)
- JNG% (Jungle Control Rate do Time Azul)
- OPP_WR (Win-Rate do Time Vermelho)
- OPP_KD (Kill-to-Death Ratio do Time Vermelho)
- OPP_GPR (gold Percent Ratio) do Time Vermelho
- OPP_GSPD (Average Gold Spent Ratio do Time Vermelho)
- OPP_EGR (Early-Game-Rate do Time Vermelho)
- OPP_MLR (Mid-Late-Game-Rate do Time Vermelho)
- OPP_FB% (First Blood Rate do Time Vermelho)
- OPP_FT% (First Blood Rate do Time Vermelho)
- OPP_F3T% (First To Three Towers Rate do Time Vermelho)
- OPP_HLD% (Harold Rate do Time Vermelho)
- OPP_DRG% (Dragon Rate do Time Vermelho)
- OPP_BN% (First Blood Rate do Time Vermelho)
- OPP_LNE% (Lane Control Rate do Time Vermelho)
- OPP_JNG% (Jungle Control Rate do Time Vermelho)

Iremos excluir, portanto, as colunas:

- golddiffat15 (Diferença de gold entre os times aos 15 minutos)
- xpdiffat15 (Diferença de XP entre os times aos 15 minutos)
- csdiffat15 (Diferença de creeps entre os times aos 15 minutos)
- killsdiffat15 (Diferença de kills entre os times aos 15 minutos)
- assistsdiffat15 (Diferença de assists entre os times aos 15 minutos)
- golddiffat10 (Diferença de gold entre os times aos 10 minutos)
- xpdiffat10 (Diferença de xp entre os times aos 10 minutos)
- csdiffat10 (Diferença de creeps entre os times aos 10 minutos)
- killsdiffat10 (Diferença de kills entre os times aos 10 minutos)
- assistsdiffat10 (Diferença de assists entre os times aos 10 minutos)

In [ ]:
# Descarte do identificador da partida
data = data.drop(columns=['id'])
# Descarte de colunas que não serão utilizadas
data = data.drop(columns= ['golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsdiffat15', 'assistsdiffat15', 'golddiffat10', 'xpdiffat10', 'csdiffat10', 'killsdiffat10', 'assistsdiffat10'])

In [ ]:
cols_to_normalize = data.columns.drop('result')

scaler = MinMaxScaler()
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])
X = data.drop(columns=['result'])
y = data['result']

# Vai armazenar os resultados dos classfiicadores (quando forem executados)
resultados = {}

## Implementação do Hetergeneos Boosting

In [ ]:


class HeterogeneousBoosting(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=1):
        self.n_estimators = n_estimators
        self.base_models = [
            GaussianNB(),  
            DecisionTreeClassifier(random_state=13),  
            MLPClassifier(random_state=13),  
            KNeighborsClassifier()  
        ]
        self.trained_models = []

    def fit(self, X, y):
         n_samples = X.shape[0]
         self.most_frequent_class = mode(y, keepdims=True).mode[0]
         sample_weights = np.ones(n_samples, dtype=float)
         self.trained_models = []
 
         for i in range(self.n_estimators):
             indices = np.random.choice(
                 np.arange(n_samples),
                 size=n_samples, 
                 replace=True,
                 p=sample_weights / sample_weights.sum()
             )
             X_resampled = X.iloc[indices]
             y_resampled = y.iloc[indices]

             best_model = None
             best_accuracy = -1
             best_model_index = None
 
             for idx, base_model in enumerate(self.base_models):
                 model = clone(base_model)
                 model.fit(X_resampled, y_resampled)
                 y_pred = model.predict(X_resampled)
 
                 accuracy = (y_pred == y_resampled).mean()
 
                 if (accuracy > best_accuracy) or (
                     accuracy == best_accuracy and self._prefer(idx, best_model_index)
                 ):
                     best_model = model
                     best_accuracy = accuracy
                     best_model_index = idx

             # print(f"Iteração {i+1}/{self.n_estimators}: Melhor modelo = {type(best_model).__name__}, Acurácia = {best_accuracy:.4f}")
             y_pred_full = best_model.predict(X)
             incorrect_mask = (y_pred_full != y)
 
             
             sample_weights[incorrect_mask] *= 2

             self.trained_models.append(best_model)




    def _prefer(self, idx1, idx2):
        # Preferência: MLP (2) > DT (1) > KNN (3) > NB (0)
        preference = [0, 1, 2, 3]
        if idx2 is None:
            return True
        return preference[idx1] > preference[idx2]


    def predict(self, X):
        n_samples = X.shape[0]
        all_preds = np.vstack([model.predict(X) for model in self.trained_models])

        final_preds = []
        for i in range(n_samples):
            
            votes = all_preds[:, i]
            vote_counts = Counter(votes)
            max_votes = max(vote_counts.values())
    
            
            most_voted_classes = [cls for cls, count in vote_counts.items() if count == max_votes]
    
            if len(most_voted_classes) == 1:
                final_preds.append(most_voted_classes[0])
            else:
                
                if self.most_frequent_class in most_voted_classes:
                    final_preds.append(self.most_frequent_class)
                else:
                    
                    final_preds.append(most_voted_classes[0])
        
        return np.array(final_preds)

    def predict_proba(self, X):
        proba_preds = []
        for model in self.trained_models:
            if hasattr(model, "predict_proba"):
                proba_preds.append(model.predict_proba(X))
        if proba_preds:
            return np.mean(proba_preds, axis=0)
        else:
            raise ValueError("Nenhum modelo no ensemble suporta `predict_proba`.")


## Função de treino-teste do modelo

In [ ]:
def train_and_evaluate_model(model, X, y):
    """
    Treina e avalia o modelo usando validação cruzada com barra de progresso.
    """
    if model == "DT":
        model = DecisionTreeClassifier(random_state=13)
        param_grid = {
            'criterion': ['gini', 'entropy'],
            'max_depth': [5, 10, 15, 25]
        }
    elif model == "KNN":
        model = KNeighborsClassifier()
        param_grid = {
            'n_neighbors':[1,3,5,7,9]
        }
    elif model == "MLP":
        model = MLPClassifier(random_state=13)
        param_grid = {
            'hidden_layer_sizes': [(100,),(10,)],
            'alpha': [0.0001, 0.005],
            'learning_rate': ['constant','adaptive']
        }
    elif model == "RF":
        model = RandomForestClassifier(random_state=13)
        param_grid = {
            'n_estimators': [5, 10, 15, 25],
            'max_depth': [10, None]
        }
    elif model == "HB":
        model = HeterogeneousBoosting()
        param_grid = {
            'n_estimators': [5, 10, 15, 25, 50]
        }
    else:
        raise ValueError(f"Modelo '{model}' não reconhecido.")

    outer_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36854321)

    scores = []

    for train_idx, test_idx in tqdm(outer_cv.split(X, y), total=outer_cv.get_n_splits(), desc=f"Validando {model.__class__.__name__}"):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        inner_cv = StratifiedKFold(n_splits=4)
        grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, scoring='accuracy', n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        scores.append(acc)

    return scores


## Funções úteis

In [ ]:
def t_corrigido_nadeau_bengio(data1, data2, X, n_folds_externos):
    """
    Parâmetros:
    data1, data2: listas ou arrays com as acurácias

    X: conjunto de dados original
    n_folds_externos: número de folds no loop externo
    Retorna:
    t_stat: valor da estatística t
    p_valor: valor-p do teste bilateral
    """
    N = len(X) # número total de amostras no dataset
    n = len(data1) # número de execuções
    # Tamanhos dos conjuntos de treino/teste em cada fold externo
    n_test = N // n_folds_externos
    n_train = N - n_test
    # Cálculo da estatística t com correção
    diffs = np.array(data1) - np.array(data2)
    mean_diff = np.mean(diffs)
    std_diff = np.std(diffs, ddof=1)
    se_corrigido = std_diff * np.sqrt(1/n + n_test/n_train)
    t_stat = mean_diff / se_corrigido
    p_valor = 2 * (1 - t.cdf(abs(t_stat), df=n - 1))
    return t_stat, p_valor

## Decision Tree (DT)

In [ ]:
scores = train_and_evaluate_model("DT", X, y)

print(f"Acurácias: {scores}")
print(f"Média: {np.mean(scores):.3f}, Desvio padrão: {np.std(scores):.3f}")

resultados["DT"] = scores


## K Nearnest Neighbor (KNN)

In [ ]:
scores = train_and_evaluate_model("KNN", X, y)

print(f"Acurácias: {scores}")
print(f"Média: {np.mean(scores):.3f}, Desvio padrão: {np.std(scores):.3f}")

resultados["KNN"] = scores

## Multi Layer Perceptron (MLP)

In [ ]:
scores = train_and_evaluate_model("MLP", X, y)

print(f"Acurácias: {scores}")
print(f"Média: {np.mean(scores):.3f}, Desvio padrão: {np.std(scores):.3f}")

resultados["MLP"] = scores

## Random Forest (RF)

In [ ]:
scores = train_and_evaluate_model("RF", X, y)

print(f"Acurácias: {scores}")
print(f"Média: {np.mean(scores):.3f}, Desvio padrão: {np.std(scores):.3f}")

resultados["RF"] = scores

## Heterogeneous Boosting (HB) 

In [ ]:
scores = train_and_evaluate_model("HB", X, y)

print(f"Acurácias: {scores}")
print(f"Média: {np.mean(scores):.3f}, Desvio padrão: {np.std(scores):.3f}")

resultados["HB"] = scores

In [ ]:


def calcular_ic95(scores):
    mean = np.mean(scores)
    std = np.std(scores, ddof=1)
    n = len(scores)
    conf = 0.95
    h = t.ppf((1 + conf) / 2, df=n - 1) * std / np.sqrt(n)
    return mean, std, mean - h, mean + h

tabela = []

for metodo, scores in resultados.items():
    media, desvio, inf, sup = calcular_ic95(scores)
    tabela.append([metodo, round(media, 3), round(desvio, 3), round(inf, 3), round(sup, 3)])
df_tabela = pd.DataFrame(tabela, columns=['Método', 'Média', 'Desvio Padrão', 'IC 95% Inferior', 'IC 95% Superior'])
df_tabela.set_index('Método', inplace=True)
df_tabela.index.name = None  
display(df_tabela)


In [ ]:


df_long = pd.DataFrame([
    {'Método': metodo, 'Acurácia': acc}
    for metodo, scores in resultados.items()
    for acc in scores
])

plt.figure(figsize=(8,6))
sns.boxplot(data=df_long, x='Método', y='Acurácia')
plt.title("Boxplot das Acurácias por Método")
plt.show()


In [ ]:


metodos = list(resultados.keys())
n = len(metodos)
matriz = pd.DataFrame("", index=metodos, columns=metodos)

for i in range(n):
    for j in range(n):
        if i == j:
            continue
        scores1 = resultados[metodos[i]]
        scores2 = resultados[metodos[j]]
        if i < j:
            stat, p = t_corrigido_nadeau_bengio(scores1, scores2, X, 10)
        else:
            
            stat, p = stats.wilcoxon(scores1, scores2)
        # Se p < 0.001, exibe 0.001
        p_show = 0.001 if p < 0.001 else p
        txt = f"**{p_show:.3f}**" if p < 0.05 else f"{p_show:.3f}"
        matriz.iloc[i, j] = txt

print("\nMatriz de Testes de Hipóteses (p-valores):")
display(matriz)
